In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [5]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self,CL):
    super(MyCLRuleMonitor,self).__init__()
    self.CL = CL


  def on_epoch_end(self,epoch,logs=None):
    testScore = logs['val_r2_score']
    trainScore = logs['r2_score']

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

In [6]:
data = pd.read_csv('50_Startups.csv')
# print(data.head())
print(data.info())
data.dropna(inplace=True)
print(data.info())

# 1. Features and Labels split
features = data.iloc[:, [0,1,2,3]].values
labels = data.iloc[:, [4]].values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB
None


In [7]:
# Transform the State values using One Hot Encoder concat it with the featues and then process

ohe = OneHotEncoder(sparse_output=False)
ohe.fit(features[:,[3]])
states = ohe.fit_transform(features[:,[3]])
print(states)
final_featues = np.concatenate((states, features[:,[0,1,2]]),axis=1)
print(final_featues)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[0.0 0.0 1.0 165349.2 136897.8 471784.1]
 [1.0 0.0 0.0 162597.7 151377.59 443898.53]
 [0.0 1.0 0.0 153441.51 101145.55 407934.54]
 [0.0 0.0 1.0 144372.41 118671.85 383199.62]
 [0.0 1.0 0.0 142107.34 91391.77 366168.42]
 [0.0 0.0 1.0 131876.9 99814.71 362861.36]
 [1.0 0.0 0.0 134615.46 147198.87 127716.82]
 [0.0 1.0 0.0 130298.13 145530.06 323876.68]
 [0.0 0.0 1.0 120542.52 148718.95 311613.29]
 

In [8]:
rs = RobustScaler()
s_features = rs.fit_transform(final_featues)

# b. Labels need to be scaled using Min Max Scaler
mn = MinMaxScaler()
m_labels = mn.fit_transform(labels)

In [9]:
# 3. Train test split
X_train, X_test, y_train, y_test = train_test_split(s_features, m_labels, test_size=0.2, random_state = 10)

In [11]:
# Deep learning model steps:
# 1. Model architeching (Creating Input/Hidden and Output layers)
# 2. Model compile
# 3. Model fit (Training)
# 4. Model evaluation
# 5. Model deployment

# https://www.tensorflow.org/api_docs/python/tf/keras/Model
# Model architecting:


model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(units = 100, activation='relu', input_shape=(6,)))
# Handle the no of feature cols and activation function as per rules.
model.add(tf.keras.layers.Dense(units= 100, activation='relu'))
model.add(tf.keras.layers.Dense(units= 100, activation='relu'))
model.add(tf.keras.layers.Dense(units= 1, activation='linear'))

# 2. Model compile:
model.compile(optimizer = 'adam',loss = 'mean_squared_error' ,metrics = [tf.keras.metrics.R2Score()])


# 3. Fit the model/ Train
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs= 100000 ,callbacks=[MyCLRuleMonitor(.85)])

Epoch 1/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 632ms/step - loss: 0.2784 - r2_score: -4.7224 - val_loss: 0.1569 - val_r2_score: -1.7275
Epoch 2/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 0.1431 - r2_score: -1.9419 - val_loss: 0.0806 - val_r2_score: -0.4009
Epoch 3/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - loss: 0.0761 - r2_score: -0.5643 - val_loss: 0.0503 - val_r2_score: 0.1252
Epoch 4/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0485 - r2_score: 0.0035 - val_loss: 0.0415 - val_r2_score: 0.2785
Epoch 5/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 0.0445 - r2_score: 0.0846 - val_loss: 0.0335 - val_r2_score: 0.4174
Epoch 6/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - loss: 0.0392 - r2_score: 0.1941 - val_loss: 0.0200 - val_r2_score: 0.6524
Epoch 7/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - loss: 0.0285 - r2_score: 0.4144 - val_loss: 0.0107 - val_r2_score: 0.8137
Epoch 8/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - loss: 0.0172 - r2_score: 0.64

In [ ]:
# 4. Model evaluation
# Evaluate the model on test data
test_loss, test_r2 = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test R2 Score: {test_r2:.4f}')

# Evaluate the model on training data
train_loss, train_r2 = model.evaluate(X_train, y_train, verbose=0)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train R2 Score: {train_r2:.4f}')

# Optional: Save the model for future use
# model.save('50_Startups_Profit_Predictor.keras')
# model.save('50_Startups_Profit_Predictor.h5')


In [ ]:

# Testing app: Reading inputs
rdspends = float(input('Enter the r&d spends: '))
adminspends = float(input('Enter the admin spends: '))
marketingspends = float(input('Enter the marketing spends: '))
state = input('Enter the location: ')

# Encode the state value:
state_values = ohe.categories_[0]
while state not in state_values:
   state = input('Enter the location: ')

# Transform this value:
input_state = ohe.transform(np.array([[state]]))
input_final_features = np.concatenate((input_state, np.array([[rdspends, adminspends, marketingspends]])), axis = 1)

# Scale the features (use transform, not fit_transform, to use the scaler fitted on training data):
scaled_input = rs.transform(input_final_features)
predicted_values = model.predict(scaled_input)

# reverse transform the predicted value:
profit  = mn.inverse_transform(predicted_values)

print(f'For values: {rdspends},{adminspends}, {marketingspends}, and {state}: Expected profit is: {profit}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
For values: 1000.0,2322.0, 4433.0, and New York: Expected profit is: [[38238.273]]


# 📊 Model Evaluation Techniques Guide

## When to Use What Evaluation Metric?

This guide covers all evaluation techniques for both **Classification** and **Regression** problems.


## 📋 Comparison Table: Evaluation Metrics

| Metric | Problem Type | Range | Best Value | When to Use | Sensitive To |
|--------|-------------|-------|------------|-------------|--------------|
| **Accuracy** | Classification | 0-1 | 1.0 | Balanced datasets, equal class importance | Class imbalance |
| **Precision** | Classification | 0-1 | 1.0 | When False Positives are costly (spam detection, fraud) | Class imbalance |
| **Recall (Sensitivity)** | Classification | 0-1 | 1.0 | When False Negatives are costly (disease detection, security) | Class imbalance |
| **F1-Score** | Classification | 0-1 | 1.0 | Need balance between Precision & Recall, imbalanced data | - |
| **ROC-AUC** | Binary Classification | 0-1 | 1.0 | Need threshold-independent metric, compare models | Class imbalance |
| **Confusion Matrix** | Classification | - | - | Understand error types, visualize performance | - |
| **MSE** | Regression | 0-∞ | 0 | Penalize large errors heavily, same units as squared target | Outliers |
| **RMSE** | Regression | 0-∞ | 0 | Same units as target, interpretable, penalize large errors | Outliers |
| **MAE** | Regression | 0-∞ | 0 | Robust to outliers, interpretable, same units as target | - |
| **R² Score** | Regression | -∞ to 1 | 1.0 | Overall model fit, percentage of variance explained | - |
| **MAPE** | Regression | 0-∞ | 0 | Percentage errors, scale-independent, business-friendly | Zero values |
| **Adjusted R²** | Regression | -∞ to 1 | 1.0 | Multiple features, penalizes unnecessary features | - |


In [ ]:
# ============================================
# REGRESSION EVALUATION METRICS - EXAMPLES
# ============================================

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt

# Sample data for regression
y_true_reg = np.array([100, 200, 300, 400, 500])
y_pred_reg = np.array([110, 190, 310, 390, 510])

print("=" * 60)
print("REGRESSION EVALUATION METRICS")
print("=" * 60)
print(f"True values: {y_true_reg}")
print(f"Predicted values: {y_pred_reg}\n")

# 1. Mean Squared Error (MSE)
mse = mean_squared_error(y_true_reg, y_pred_reg)
print(f"1. MSE (Mean Squared Error): {mse:.2f}")
print("   → Use when: You want to penalize large errors heavily")
print("   → Best for: Models where large errors are unacceptable")
print("   → Range: 0 to ∞ (lower is better)\n")

# 2. Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"2. RMSE (Root Mean Squared Error): {rmse:.2f}")
print("   → Use when: You want interpretable errors in same units as target")
print("   → Best for: General regression problems, easy to explain")
print("   → Range: 0 to ∞ (lower is better)\n")

# 3. Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true_reg, y_pred_reg)
print(f"3. MAE (Mean Absolute Error): {mae:.2f}")
print("   → Use when: You want robustness to outliers")
print("   → Best for: When outliers are present, interpretable errors")
print("   → Range: 0 to ∞ (lower is better)\n")

# 4. R² Score (Coefficient of Determination)
r2 = r2_score(y_true_reg, y_pred_reg)
print(f"4. R² Score: {r2:.4f}")
print("   → Use when: You want to know % of variance explained")
print("   → Best for: Overall model quality assessment")
print("   → Range: -∞ to 1 (1.0 = perfect, 0 = baseline, negative = worse than baseline)\n")

# 5. Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_true_reg, y_pred_reg) * 100
print(f"5. MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print("   → Use when: You need percentage-based errors for business")
print("   → Best for: Business reporting, scale-independent comparison")
print("   → Range: 0 to ∞% (lower is better)\n")

# 6. Adjusted R² (for multiple features)
# Formula: 1 - [(1-R²)(n-1)/(n-k-1)] where n=samples, k=features
n = len(y_true_reg)
k = 1  # number of features (example)
adj_r2 = 1 - ((1 - r2) * (n - 1) / (n - k - 1))
print(f"6. Adjusted R²: {adj_r2:.4f}")
print("   → Use when: Comparing models with different numbers of features")
print("   → Best for: Multiple regression, feature selection")
print("   → Range: -∞ to 1 (penalizes unnecessary features)\n")


In [ ]:
# ============================================
# CLASSIFICATION EVALUATION METRICS - EXAMPLES
# ============================================

from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, roc_auc_score, confusion_matrix, 
                            classification_report, roc_curve)
from sklearn.preprocessing import label_binarize

# Sample data for binary classification
y_true_binary = np.array([0, 1, 1, 0, 1, 0, 0, 1, 1, 1])
y_pred_binary = np.array([0, 1, 1, 0, 0, 0, 1, 1, 1, 1])
y_proba_binary = np.array([0.1, 0.9, 0.8, 0.2, 0.4, 0.3, 0.6, 0.85, 0.95, 0.9])

print("=" * 60)
print("BINARY CLASSIFICATION EVALUATION METRICS")
print("=" * 60)
print(f"True labels: {y_true_binary}")
print(f"Predicted labels: {y_pred_binary}")
print(f"Predicted probabilities: {y_proba_binary}\n")

# 1. Accuracy
accuracy = accuracy_score(y_true_binary, y_pred_binary)
print(f"1. Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print("   → Use when: Classes are balanced, all errors are equally important")
print("   → Best for: Balanced datasets, general performance measure")
print("   → Range: 0 to 1 (higher is better)\n")

# 2. Precision
precision = precision_score(y_true_binary, y_pred_binary)
print(f"2. Precision: {precision:.4f}")
print("   → Use when: False Positives are costly (spam detection, fraud)")
print("   → Best for: When you can't afford false alarms")
print("   → Range: 0 to 1 (higher is better)\n")

# 3. Recall (Sensitivity)
recall = recall_score(y_true_binary, y_pred_binary)
print(f"3. Recall (Sensitivity): {recall:.4f}")
print("   → Use when: False Negatives are costly (disease detection, security)")
print("   → Best for: When you can't afford to miss positive cases")
print("   → Range: 0 to 1 (higher is better)\n")

# 4. F1-Score
f1 = f1_score(y_true_binary, y_pred_binary)
print(f"4. F1-Score: {f1:.4f}")
print("   → Use when: Need balance between Precision & Recall")
print("   → Best for: Imbalanced datasets, harmonic mean of P & R")
print("   → Range: 0 to 1 (higher is better)\n")

# 5. ROC-AUC Score
roc_auc = roc_auc_score(y_true_binary, y_proba_binary)
print(f"5. ROC-AUC Score: {roc_auc:.4f}")
print("   → Use when: Need threshold-independent metric")
print("   → Best for: Comparing models, class imbalance")
print("   → Range: 0 to 1 (1.0 = perfect, 0.5 = random)\n")

# 6. Confusion Matrix
cm = confusion_matrix(y_true_binary, y_pred_binary)
print(f"6. Confusion Matrix:")
print(f"   {cm}")
print("   → Use when: You need to understand error types")
print("   → Best for: Visualizing TP, TN, FP, FN")
print("   → Format: [[TN, FP], [FN, TP]]\n")


In [ ]:
# ============================================
# MULTICLASS CLASSIFICATION - EXAMPLES
# ============================================

# Sample data for multiclass classification (3 classes: 0, 1, 2)
y_true_multi = np.array([0, 1, 2, 0, 1, 2, 0, 1, 2])
y_pred_multi = np.array([0, 1, 2, 0, 1, 1, 0, 2, 2])

print("=" * 60)
print("MULTICLASS CLASSIFICATION EVALUATION METRICS")
print("=" * 60)
print(f"True labels: {y_true_multi}")
print(f"Predicted labels: {y_pred_multi}\n")

# For multiclass, we can use different averaging strategies
print("1. Accuracy (Multiclass):")
acc_multi = accuracy_score(y_true_multi, y_pred_multi)
print(f"   Accuracy: {acc_multi:.4f}\n")

print("2. Precision (Multiclass - Macro Average):")
prec_macro = precision_score(y_true_multi, y_pred_multi, average='macro')
print(f"   Macro Precision: {prec_macro:.4f}")
print("   → Average of precision for each class (equal weight)\n")

print("3. Precision (Multiclass - Weighted Average):")
prec_weighted = precision_score(y_true_multi, y_pred_multi, average='weighted')
print(f"   Weighted Precision: {prec_weighted:.4f}")
print("   → Weighted by number of samples per class\n")

print("4. Recall (Multiclass - Macro Average):")
recall_macro = recall_score(y_true_multi, y_pred_multi, average='macro')
print(f"   Macro Recall: {recall_macro:.4f}\n")

print("5. F1-Score (Multiclass - Macro Average):")
f1_macro = f1_score(y_true_multi, y_pred_multi, average='macro')
print(f"   Macro F1: {f1_macro:.4f}\n")

print("6. Classification Report:")
print(classification_report(y_true_multi, y_pred_multi))
print("   → Provides precision, recall, F1 for each class\n")


## 🎯 Quick Decision Guide

### For REGRESSION Problems:

**Use MSE/RMSE when:**
- You want to penalize large errors heavily
- Same units as target variable needed
- Example: House price prediction, stock price prediction

**Use MAE when:**
- You have outliers in your data
- You want interpretable errors
- Example: Salary prediction with some extreme values

**Use R² when:**
- You want to know overall model quality
- You need to compare different models
- Example: Comparing multiple regression models

**Use MAPE when:**
- You need percentage-based errors
- Business stakeholders need easy interpretation
- Example: Sales forecasting, revenue prediction

### For CLASSIFICATION Problems:

**Use Accuracy when:**
- Classes are balanced
- All errors are equally important
- Example: Balanced email classification

**Use Precision when:**
- False Positives are costly
- Example: Spam detection (don't want to mark important emails as spam)

**Use Recall when:**
- False Negatives are costly
- Example: Disease detection (don't want to miss a disease)

**Use F1-Score when:**
- You need balance between Precision & Recall
- Classes are imbalanced
- Example: Fraud detection, medical diagnosis

**Use ROC-AUC when:**
- You want threshold-independent metric
- You need to compare multiple models
- Example: Model selection, binary classification with imbalanced data

**Use Confusion Matrix when:**
- You need to understand error types
- You want to visualize performance
- Example: Understanding where model fails


In [ ]:
# ============================================
# REAL-WORLD USE CASE EXAMPLES
# ============================================

print("=" * 60)
print("REAL-WORLD USE CASE EXAMPLES")
print("=" * 60)

print("\n📊 REGRESSION USE CASES:\n")

print("1. HOUSE PRICE PREDICTION")
print("   → Use: RMSE or MAE")
print("   → Why: Easy to interpret (dollars), penalize large errors")
print("   → Example: 'Average prediction error is $50,000'\n")

print("2. SALARY PREDICTION")
print("   → Use: MAE or R²")
print("   → Why: Robust to outliers, overall model quality")
print("   → Example: 'Model explains 85% of salary variance'\n")

print("3. SALES FORECASTING")
print("   → Use: MAPE")
print("   → Why: Business-friendly percentage errors")
print("   → Example: 'Average error is 12% of actual sales'\n")

print("4. STOCK PRICE PREDICTION")
print("   → Use: RMSE or MAE")
print("   → Why: Same units, interpretable")
print("   → Example: 'Average error is $2.50 per share'\n")

print("\n🎯 CLASSIFICATION USE CASES:\n")

print("1. SPAM EMAIL DETECTION")
print("   → Use: Precision (or F1-Score)")
print("   → Why: Can't mark important emails as spam (False Positives costly)")
print("   → Example: '99% of emails marked as spam are actually spam'\n")

print("2. DISEASE DETECTION (Medical)")
print("   → Use: Recall (or F1-Score)")
print("   → Why: Can't miss detecting a disease (False Negatives costly)")
print("   → Example: 'Detects 95% of all actual diseases'\n")

print("3. FRAUD DETECTION")
print("   → Use: F1-Score or ROC-AUC")
print("   → Why: Need balance, imbalanced data (few fraud cases)")
print("   → Example: 'Balanced performance on both precision and recall'\n")

print("4. CUSTOMER CHURN PREDICTION")
print("   → Use: F1-Score or ROC-AUC")
print("   → Why: Imbalanced data, need balance")
print("   → Example: 'Good at identifying customers who will churn'\n")

print("5. IMAGE CLASSIFICATION (Multiclass)")
print("   → Use: Accuracy or Macro F1-Score")
print("   → Why: Multiple classes, need overall performance")
print("   → Example: 'Correctly classifies 92% of images'\n")


In [ ]:
# ============================================
# TENSORFLOW/KERAS EVALUATION METRICS
# ============================================

print("=" * 60)
print("TENSORFLOW/KERAS EVALUATION METRICS")
print("=" * 60)

print("\nFor REGRESSION in TensorFlow/Keras:\n")
print("Available metrics:")
print("  - 'mse' or tf.keras.metrics.MeanSquaredError()")
print("  - 'mae' or tf.keras.metrics.MeanAbsoluteError()")
print("  - 'rmse' (custom) or tf.keras.metrics.RootMeanSquaredError()")
print("  - tf.keras.metrics.R2Score()")
print("\nExample:")
print("  model.compile(optimizer='adam',")
print("                loss='mean_squared_error',")
print("                metrics=[tf.keras.metrics.R2Score(),")
print("                         tf.keras.metrics.MeanAbsoluteError()])")

print("\n" + "-" * 60)
print("\nFor CLASSIFICATION in TensorFlow/Keras:\n")
print("Available metrics:")
print("  - 'accuracy' or tf.keras.metrics.Accuracy()")
print("  - 'precision' or tf.keras.metrics.Precision()")
print("  - 'recall' or tf.keras.metrics.Recall()")
print("  - tf.keras.metrics.AUC()")
print("  - tf.keras.metrics.F1Score()")
print("\nExample:")
print("  model.compile(optimizer='adam',")
print("                loss='binary_crossentropy',")
print("                metrics=['accuracy',")
print("                         tf.keras.metrics.Precision(),")
print("                         tf.keras.metrics.Recall(),")
print("                         tf.keras.metrics.AUC()])")

print("\n" + "=" * 60)
print("\n💡 TIP: Use model.evaluate() to get metrics on test data")
print("   Example: loss, accuracy = model.evaluate(X_test, y_test)")
print("=" * 60)


## 🔀 Decision Flowchart

### REGRESSION Metrics Decision Tree:

```
Is your target variable continuous/numeric?
    │
    ├─ YES → REGRESSION
    │   │
    │   ├─ Need percentage errors for business?
    │   │   ├─ YES → Use MAPE
    │   │   └─ NO → Continue...
    │   │
    │   ├─ Have outliers in data?
    │   │   ├─ YES → Use MAE (robust to outliers)
    │   │   └─ NO → Continue...
    │   │
    │   ├─ Need to penalize large errors heavily?
    │   │   ├─ YES → Use RMSE or MSE
    │   │   └─ NO → Use MAE
    │   │
    │   └─ Need overall model quality assessment?
    │       └─ YES → Use R² Score
```

### CLASSIFICATION Metrics Decision Tree:

```
Is your target variable categorical?
    │
    ├─ YES → CLASSIFICATION
    │   │
    │   ├─ How many classes?
    │   │   ├─ 2 classes → BINARY CLASSIFICATION
    │   │   │   │
    │   │   │   ├─ Classes balanced?
    │   │   │   │   ├─ YES → Use Accuracy
    │   │   │   │   └─ NO → Continue...
    │   │   │   │
    │   │   │   ├─ False Positives costly? (spam, fraud alerts)
    │   │   │   │   └─ YES → Use Precision
    │   │   │   │
    │   │   │   ├─ False Negatives costly? (disease, security)
    │   │   │   │   └─ YES → Use Recall
    │   │   │   │
    │   │   │   ├─ Need balance between P & R?
    │   │   │   │   └─ YES → Use F1-Score
    │   │   │   │
    │   │   │   └─ Need threshold-independent metric?
    │   │   │       └─ YES → Use ROC-AUC
    │   │   │
    │   │   └─ 3+ classes → MULTICLASS CLASSIFICATION
    │   │       │
    │   │       ├─ Classes balanced?
    │   │       │   ├─ YES → Use Accuracy or Macro F1
    │   │       │   └─ NO → Use Weighted F1 or Macro F1
    │   │       │
    │   │       └─ Need per-class metrics?
    │   │           └─ YES → Use Classification Report
```

## 📝 Summary

**Key Takeaways:**
1. **Regression**: Use RMSE/MAE for interpretability, R² for overall quality, MAPE for business reporting
2. **Binary Classification**: Use Precision when FP costly, Recall when FN costly, F1 for balance, ROC-AUC for comparison
3. **Multiclass**: Use Accuracy for balanced, Macro/Weighted F1 for imbalanced
4. **Always consider**: Class imbalance, cost of errors, business requirements
